In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

In [36]:
df = pd.read_csv('https://raw.githubusercontent.com/DzhamStriker/Assotiation-Rules/main/dataset.csv')

In [37]:
df

,0,1,2,3,4,5,6
0,Bread,Wine,Eggs,Meat,Cheese,Pencil,Diaper
1,Bread,Cheese,Meat,Diaper,Wine,Milk,Pencil
2,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
3,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
4,Meat,Pencil,Wine,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
310,Bread,Eggs,Cheese,NaN,NaN,NaN,NaN
311,Meat,Milk,Pencil,NaN,NaN,NaN,NaN
312,Bread,Cheese,Eggs,Meat,Pencil,Diaper,Wine
313,Meat,Cheese,NaN,NaN,NaN,NaN,NaN


In [38]:
items = (df['0'].unique())
items

array(['Bread', 'Cheese', 'Meat', 'Eggs', 'Wine', 'Bagel', 'Pencil',
       'Diaper', 'Milk'], dtype=object)

In [39]:
itemset = set(items)
encoded_vals = []
for index, row in df.iterrows():
    rowset = set(row) 
    labels = {}
    uncommons = list(itemset - rowset)
    commons = list(itemset.intersection(rowset))
    for uc in uncommons:
        labels[uc] = 0
    for com in commons:
        labels[com] = 1
    encoded_vals.append(labels)
encoded_vals[0]
ohe_df = pd.DataFrame(encoded_vals)

In [40]:
support = np.array([0 for i in range(9)])
for index, row in ohe_df.iterrows():
    s = np.array(row)
    support += np.array(row)

In [41]:
df2 = list(ohe_df.columns.values)
#for i in support
df2

['Bagel',
 'Milk',
 'Bread',
 'Wine',
 'Meat',
 'Cheese',
 'Eggs',
 'Diaper',
 'Pencil']

In [42]:
support2 = dict(zip(df2, support))

In [43]:
from itertools import chain, combinations

In [44]:
def return_min_support(items, transactions, min_support, freqSet):
    items_t = set()
    localSet = defaultdict(int)
    for i in items:
        #print(i)
        for index, row in transactions.iterrows():
            bool_in = 1
            for item in i:
                #print(item, row[item])
                if row[item] == 0:
                    bool_in = 0
            if bool_in:
                freqSet[i] += 1
                localSet[i] += 1
    for item, count in localSet.items():
        support = float(count) / len(transactions)
        if support > min_support:
            items_t.add(item)
    return items_t

In [45]:
min_support = 0.2
F = []
F1 = set()
for i in range(len(support2)):
    if support[i] >= min_support:
        F1.add(frozenset([df2[i]]))
current_set = F1
large_set = dict()
k = 2
#print(1)
rules = dict()
freqSet = defaultdict(int)
for i in F1:
    freqSet[i] = support2[list(i)[0]]
#print(current_set)
while (len(current_set) != 0):
    large_set[k - 1] = current_set
    current_set = set(i.union(j) for i in current_set for j in current_set if len(i.union(j)) == k)
    #print(current_set)
    next_set = return_min_support(current_set, ohe_df, min_support, freqSet)
    current_set = next_set
    k += 1

In [46]:
def subsets(arr):
    return chain(*[combinations(arr, i + 1) for i, a in enumerate(arr)])
def getSupport(item):
        """local function which Returns the support of an item"""
        return float(freqSet[item]) / len(ohe_df)

In [47]:
#getSupport(F1[0])
#getSupport(list(F1)[0])

In [48]:
toRetItems = []
for key, value in large_set.items():
    toRetItems.extend([(tuple(item), getSupport(item)) for item in value])

#проверка на confidence
min_confidence = 1000
toRetRules = []
for key, value in list(large_set.items())[1:]:
    for item in value:
        _subsets = map(frozenset, [x for x in subsets(item)])
        for element in _subsets:
            remain = item.difference(element)
            if len(remain) > 0:
                confidence = getSupport(item) / getSupport(element)
                if confidence >= min_confidence:
                    print(confidence)
                    toRetRules.append(((tuple(element), tuple(remain)), confidence))
print(toRetItems)

[(('Diaper',), 0.40634920634920635), (('Wine',), 0.4380952380952381), (('Milk',), 0.5015873015873016), (('Bagel',), 0.4253968253968254), (('Eggs',), 0.4380952380952381), (('Bread',), 0.5047619047619047), (('Pencil',), 0.3619047619047619), (('Cheese',), 0.5015873015873016), (('Meat',), 0.47619047619047616), (('Diaper', 'Bread'), 0.23174603174603176), (('Wine', 'Milk'), 0.21904761904761905), (('Eggs', 'Cheese'), 0.2984126984126984), (('Bagel', 'Milk'), 0.2253968253968254), (('Meat', 'Cheese'), 0.3238095238095238), (('Bread', 'Wine'), 0.24444444444444444), (('Bread', 'Milk'), 0.27936507936507937), (('Eggs', 'Milk'), 0.24444444444444444), (('Bagel', 'Bread'), 0.27936507936507937), (('Wine', 'Cheese'), 0.2698412698412698), (('Bread', 'Cheese'), 0.23809523809523808), (('Meat', 'Wine'), 0.2507936507936508), (('Eggs', 'Wine'), 0.24126984126984127), (('Milk', 'Cheese'), 0.3047619047619048), (('Diaper', 'Wine'), 0.23492063492063492), (('Meat', 'Milk'), 0.24444444444444444), (('Bread', 'Meat'), 0